# Project-2: Solving the Poisson-Boltzman Equation 

### System of two non-linear equations

In [1]:
using NLsolve

function f!(F, x)
    F[1] = (x[1]+3)*(x[2]^3-7)+18
    F[2] = sin(x[2]*exp(x[1])-1)
end

function j!(J, x)
    J[1, 1] = x[2]^3-7
    J[1, 2] = 3*x[2]^2*(x[1]+3)
    u = exp(x[1])*cos(x[2]*exp(x[1])-1)
    J[2, 1] = x[2]*u
    J[2, 2] = u
end

nlsolve(f!, j!, [ 0.1; 1.2])

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.1, 1.2]
 * Zero: [-3.7818e-16, 1.0]
 * Inf-norm of residuals: 0.000000
 * Iterations: 4
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 5
 * Jacobian Calls (df/dx): 5

### System of three linear equations

In [7]:
using NLsolve

function f!(F, x)
    F[1] = x[1]-0
    F[2] = 4*x[1]-8*x[2]+4*x[3] + x[2]*x[2]
    F[3] = x[3]-1
end

function j!(J, x)
    J[1, 1] = 1
    J[1, 2] = 0
    J[1, 3] = 0
    J[2, 1] = 4
    J[2, 2] = -8 + x[2]
    J[2, 3] = 4 
    J[3, 1] = 0 
    J[3, 2] = 0
    J[3, 3] = 1
end

nlsolve(f!, j!, [ 1.; 1. ; 1.])

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [1.0, 1.0, 1.0]
 * Zero: [0.0, 0.535898, 1.0]
 * Inf-norm of residuals: 0.000000
 * Iterations: 8
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 9
 * Jacobian Calls (df/dx): 9

## System with many unknowns from discourse

The example that follows was taken from [https://discourse.julialang.org/t/correct-use-of-the-sparsity-pattern-in-nlsolve-jl-for-diagonal-jacobians/43469] . In this example, the Jacobian is diagonal and computed analytically. 

In [4]:
using SparseArrays
using NLsolve
using SpecialFunctions

Ne = 10

function f_sparse!(F, x)
    @inbounds for (i, xi) in enumerate(x)
        F[i] = exp(-xi^2/(2*i))*erf(xi)
    end 
end

function g_sparse!(J, x)
    @inbounds for (i, xi) in enumerate(x)
        J[i,i] = (-xi/i)*exp(-xi^2/(2*i))*erf(xi) + exp(-xi^2/(2*i))*(2/√π)*exp(-xi^2)
    end 
end

df = OnceDifferentiable(f_sparse!, g_sparse!, randn(Ne), rand(Ne), spzeros(Ne, Ne))

F = rand(Ne);

r = nlsolve(df, F, method = :newton);

r

Results of Nonlinear Solver Algorithm
 * Algorithm: Newton with line-search
 * Starting Point: [0.42763083949335656, 0.7443300066041312, 0.9487526292345498, 0.8710385079998757, 0.9974234850269958, 0.06265105653918313, 0.46595920639015387, 0.629224859060509, 0.6347382091329619, 0.7208091418492224]
 * Zero: [0.0, -10.974479854497067, -10.767607940296372, -15.256373327103237, -13.641530100343765, 0.0, 0.0, 0.0, 0.0, 0.0]
 * Inf-norm of residuals: 0.000000
 * Iterations: 18
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 19
 * Jacobian Calls (df/dx): 19

## Vector-Valued Example: PDE discretized on coarse spatial mesh

In [ ]:
h = 1/3; h2 = h*h;
h = 1; h2 = h*h;
#..observe that matrix is zero is first and last row to preserve the boundary conditions 
A = [0. 0 0 0; 1/h2 -2/h2 1/h2 0; 0 1/h2 -2/h2 1/h2; 0 0 0 0.]

function wave_system!(du,u,p,t)
    du = A*u 
end

#..fix initial velocities
v0 = zeros(4,1)
v0[2]=1; v0[3]=-1; 

#..fix initial positions 
u0 = zeros(4,1); 
u0[1]=0; u0[2]=2; u0[3]=2; u0[4]=1;

tspan = (0.0,2.0)               

prob = SecondOrderODEProblem(wave_system!,v0,u0,tspan)
sol = solve(prob,DPRKN6(),reltol=1e-8,abstol=1e-8)

plot(sol,vars=5,label="Pos-1")
p1=plot!(sol,vars=8,label="Pos-4")

plot(sol,vars=6,label="Pos-2")
p2=plot!(sol,vars=7,label="Pos-3")

plot(p1,p2,layout=(2,1))